In [2]:
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import XMLOutputParser, JsonOutputParser, PydanticOutputParser

load_dotenv()

True

In [3]:
model = ChatGroq(model="gemma2-9b-it")

In [4]:
from pydantic import Field, BaseModel
from typing import Optional

class Product(BaseModel):
    name: str = Field(description="The name of the product")
    tentative_price: Optional[float] = Field(default=None, description="The price of the product in USD")
    description: str = Field(description="A brief description of the product")


In [5]:
from langchain_core.prompts import  ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage

product_parser = PydanticOutputParser(pydantic_object=Product)

_prompt_template = PromptTemplate(
    template="You are a helpful eCommerce Product assistant. Answer the user question: {input}. Return the product information in the format: {format_instructions}",
    input_variables=["input"],
    partial_variables={"format_instructions": product_parser.get_format_instructions()}
)

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=_prompt_template),
        HumanMessage("input"),
    ]
)

chain = prompt | model | product_parser

response = chain.invoke({"input": "Tell me about the latest iPhone."})
response

Product(name='iPhone 15 Pro Max', tentative_price=1099.0, description='The latest iPhone, featuring a stunning 6.7-inch Super Retina XDR display, a powerful A17 Pro chip, a revolutionary titanium design, and an advanced camera system with a new 48MP main camera.')